In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr,spearmanr, ttest_ind, ttest_rel, mannwhitneyu, shapiro, wilcoxon
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore") # ignore a warning related to tsplot()


In [36]:
# construct the matrix for result consistency between single dataset and the meta results with all the other datasets.
dataset_info_file = './doc/dataset_info.csv'
dataset_info = pd.read_csv(dataset_info_file)
#dataset_info = dataset_info[dataset_info['ScannerField0']=='1.5'] # 1.5/3
#folder_str = 'Scanner3'
#folder_str = 'Scanner1.5'
#dataset_info = dataset_info[dataset_info['AgeMax']<=18]
#folder_str = 'Child'
#dataset_info = dataset_info[dataset_info['AgeMin']>18]
#folder_str = 'Adult'
dataset_info = dataset_info[~dataset_info['Dataset'].isin(['BIG', 'QTIM'])] # check results after exluding the largest two datasets
folder_str = 'exclBIGQTIM'

pthr_single = 0.05 # significance threshold
pthr = 0.05 # significance threshold for meta
#pthr = 0.05/70.0# meta threshold when considering multiple testing correction

dataset_by_region_mat = np.zeros((len(dataset_info['Dataset']),70)) 
# 1 for the same results (sig. & same direction, or all not sig.), otherwise 0 

dataset_by_region_mat2 = np.zeros((len(dataset_info['Dataset']),70)) 
# 1 for the same results (sig. & same direction), otherwise 0 

for j, dataset in enumerate(dataset_info['Dataset']):
    print(dataset)
    # read the meta results while excluding that dataset
    #meta_f = u'./meta/sum_asy_meta_excl_'+dataset+'.csv'
    meta_f = u'./meta_subgrp/'+folder_str+'/sum_asy_meta_excl_'+dataset+'.csv'
    
    meta_dat = pd.read_csv(meta_f)
    meta_dat[u'estimate_abs'] = np.abs(meta_dat[u'estimate'])
    
    csv_list =  meta_dat[u'Region']
    for i, csv_f in enumerate(csv_list):
        meta_z = meta_dat[u'zval'][i] # meta effect for that region
        meta_p = meta_dat[u'pval'][i] # meta effect for that region
        csv_dat = pd.read_csv(os.path.join('./data/',csv_f))
        
        csv_dat = csv_dat[csv_dat['Dataset']==dataset]
        #print(meta_p, meta_z, csv_dat[u'p'].values[0], csv_dat[u't'].values[0])
        if (meta_p <=pthr) and (csv_dat[u'p'].values[0]<=pthr_single) and ((csv_dat[u't'].values[0]*meta_z)>0)==True: 
            # significant & same direction
            dataset_by_region_mat[j,i] = 1
        elif (meta_p >pthr) and (csv_dat[u'p'].values[0]>pthr_single): # both not sig.
            dataset_by_region_mat[j,i] = 1
        else: # meta_p > pthr for ~false positive
            dataset_by_region_mat[j,i] = 0 
            
        if (meta_p <=pthr) and (csv_dat[u'p'].values[0]<=pthr_single) and ((csv_dat[u't'].values[0]*meta_z)>0)==True: 
            # significant & same direction
            dataset_by_region_mat2[j,i] = 1            
        else: # meta_p > pthr for ~false positive
            dataset_by_region_mat2[j,i] = 0

#plt.imshow(dataset_by_region_mat)
#plt.show()

#plt.imshow(dataset_by_region_mat2)
#plt.show()


ACPU
ADHD-DUB1
ADHD WUE
ADHD_Rubia1
ADHD200_KKI
ADHD200_NYU
ADHD200_OHSU
ADHD200_Peking
ADHDKonrad
Bergen_adultADHD
Bergen_SVG
CAPS_UZH
DAT_london
Dundee
Hartford_Olin
IMpACT_NL
MGH_ADHD
MTA
NeuroImage_ADAM
NeuroImage_NIJM
NICAP
NICHE
NIH
NYU ADHD
OHSU
UAB-ADHD
UCHZ
HUBIN_KASP
TOP15T
TOP3T_0
TOP3T_GE750
Anna
AnneU
BILGIN
CAMH_ASD
Christoph
ClarissaBr
Colm_UCSF
ClinG_sample
HMS_sample
EstherCOBRE
EstherMCIC
FIDMAG
GEB
GloriaBPSydney
JPCapeTown
Malt
MatthewSacchet
MDD_Ilya
MichelleLBC
ASY_results_FOR2107
ASY_results_MuensterCohort
NESDA
NeuroIMAGE
NicolaMAS
NicolaOATS
NORM_Moscow
NUIG
01_Cheng_1.5T
01_Cheng_3T
02_Heuvel_1.5T
02_Heuvel_3T
03_Huyser
04_Mataix_Cols
OXUK
GRADUAL
OLDERS
Quinn
SaoPaulo1
SaoPaulo3
SaudEPIGEN
SHIP-2
SHIP-TREND-0
StenerEOP
UMCG_sample_groenewold
1.5T_Adults_Lateralization
3T_Adults_Lateralization
3T_Child_Adolescent_Lateralization
Cousijn
DStein
EStein
Foxe
Garavan
London
Luijten
Momenan
Orr
Ozlem
Paulus
Schmaal
Sinha
Sjoerds
VanHolst
Yucel
Kwon_3T
KwonNMC_15T
Kw

In [37]:
# characterize the reproducibility
#dataset_info_file = './doc/dataset_info.csv'
#dataset_info = pd.read_csv(dataset_info_file)

print('---Overall---')
print('Ovall mean and sd, across 70 region effects\n', 
      dataset_by_region_mat.mean(axis=0).mean(), 
      dataset_by_region_mat.mean(axis=0).std())
print('Max, and min:\n',
      dataset_by_region_mat.mean(axis=0).min(),
      dataset_by_region_mat.mean(axis=0).max())
print('---Total, and Regional---')
print('Total surface asymmetry',dataset_by_region_mat[:,0].mean(axis=0).mean())
print('Average thickness asymmetry',dataset_by_region_mat[:,1].mean(axis=0).mean())


---Overall---
Ovall mean and sd, across 70 region effects
 0.6265095729013253 0.23330599711114683
Max, and min:
 0.1958762886597938 0.9690721649484536
---Total, and Regional---
Total surface asymmetry 0.6597938144329897
Average thickness asymmetry 0.35051546391752575
